In [25]:
import setup_runs as sr
import os, os.path
import pandas as pd
import numpy as np
import pyDOE as pyd

#export files?
export_ed_files_q = True



##########################################
###                                    ###
###    GENERATE EXPERIMENTAL DESIGN    ###
###                                    ###
##########################################

print("Starting build of components for experimental design.")


#number of lhs samples
n_lhs = sr.dict_init["n_lhs"]
#set baseline strategy
strat_baseline = 0#[x for x in sr.dict_strat.keys() if sr.dict_strat[x]["strategy_id"] == 0][0]

###   SOME ATTRIBUTE TABLES
df_attribute_design_id = pd.read_csv(sr.fp_csv_attribute_design)
#reduce
df_attribute_design_id = df_attribute_design_id[df_attribute_design_id["include"] == 1]
#get time series id
df_attribute_time_series_id = pd.read_csv(sr.fp_csv_attribute_time_series)


print("Check: export_ed_files_q = " + str(export_ed_files_q))
print("Check: n_lhs = " + str(n_lhs))




#################################
#    MULTI SECTOR COMPONENTS    #
#################################


###   GET THE PARAMETER TABLE

#read in uncertainty table for additional sectors
parameter_table_additional_sectors = pd.read_csv(sr.fp_csv_parameter_ranges)
#reduce
parameter_table_additional_sectors = parameter_table_additional_sectors[parameter_table_additional_sectors["type"].isin(["incertidumbre", "accion"])]
#add field
parameter_table_additional_sectors["variable_name_lower"] = [x.lower().replace(" ", "_") for x in list(parameter_table_additional_sectors["parameter"])]
#fill nas and set to integer
parameter_table_additional_sectors["parameter_constant_q"] = parameter_table_additional_sectors["parameter_constant_q"].fillna(0)
parameter_table_additional_sectors["parameter_constant_q"] = np.array(parameter_table_additional_sectors["parameter_constant_q"]).astype(int)
#initialize available groups
groups_norm = set([int(x) for x in parameter_table_additional_sectors["normalize_group"] if not np.isnan(x)])

# IDENTIFY PARAMETERS THAT DO NOT VARY
df_apn = parameter_table_additional_sectors[["variable_name_lower", "min_2050", "max_2050"]].copy().drop_duplicates().reset_index(drop = True)
#initialize the set
all_params_novary = set({})
#loop to build
for p in df_apn["variable_name_lower"].unique():
    df_apn_tmp = df_apn[df_apn["variable_name_lower"] == p]
    #check to see if it doesn't vary
    if len(df_apn_tmp) == 1:
        if float(df_apn_tmp["min_2050"].iloc[0]) == float(df_apn_tmp["max_2050"].iloc[0]):
            if float(df_apn_tmp["min_2050"].iloc[0]) == 1.0:
                all_params_novary = all_params_novary | set({p})



###   NORMALIZATION GROUP IDS

group_id = parameter_table_additional_sectors[["type", "parameter", "normalize_group"]].drop_duplicates()
#build normalize group and lever group ids (for deltas)
norm_vec = []

if len(groups_norm) > 0:
    #starting point for new group
    ind_group = max(groups_norm) + 1
else:
    ind_group = 1

#loop over rows
for i in range(0, len(group_id)):
    #get current group
    group_cur = group_id["normalize_group"].iloc[i]
    #test for NaN
    if np.isnan(group_cur):
        norm_vec = norm_vec + [ind_group]
        #next iteration
        ind_group += 1
    else:
        norm_vec = norm_vec + [int(group_cur)]

#add to group_id data frame
group_id["norm_group_id"] = norm_vec
group_id = group_id.reset_index(drop = True)


###   LEVER GROUP IDS

#temporary data frame to build lever group id with
group_id_tmp = group_id[["type", "norm_group_id"]].drop_duplicates()
group_id_tmp = group_id_tmp[group_id_tmp["type"].isin(["Accion", "accion"])]
group_id_tmp["lever_group_id"] = range(1, len(group_id_tmp) + 1)
if "lever_group_id" not in group_id.columns:
    group_id = pd.merge(group_id, group_id_tmp[["norm_group_id", "lever_group_id"]], how = "left", left_on = ["norm_group_id"], right_on = ["norm_group_id"])
#reduce
group_id = group_id[["parameter", "type", "norm_group_id", "lever_group_id"]]
#replace nas
group_id = group_id.fillna({"lever_group_id": -1})
#integer
group_id["lever_group_id"] = [int(x) for x in list(group_id["lever_group_id"])]
#merge in
group_id  = pd.merge(group_id, parameter_table_additional_sectors[["parameter", "variable_name_lower"]], how = "left", left_on = ["parameter"], right_on = ["parameter"])
group_id = group_id.drop_duplicates()
group_id = group_id[["parameter", "type", "variable_name_lower", "norm_group_id", "lever_group_id"]]


###   MERGE BACK IN TO PTAS AND CLEAN NAMES/DATA

#set merge fields and merge
fields_merge = list(set(parameter_table_additional_sectors.columns).intersection(set(group_id.columns)))
parameter_table_additional_sectors = pd.merge(parameter_table_additional_sectors, group_id, how = "left", left_on = fields_merge, right_on = fields_merge)
#fields to extract
fields_ext = [x for x in parameter_table_additional_sectors if (x not in ["normalize_group"])]
#clean the data frame
parameter_table_additional_sectors = parameter_table_additional_sectors[fields_ext]
parameter_table_additional_sectors = parameter_table_additional_sectors.dropna(axis = 1)
#dictionary to rename
dict_ptas_rename = dict([[x, x.lower().replace(" ", "_")] for x in parameter_table_additional_sectors.columns])
#set type
parameter_table_additional_sectors = parameter_table_additional_sectors.rename(columns = dict_ptas_rename)

#parameters to index
fields_add_sec_all_vals = ["strategy_id", "type", "parameter", "sector", "norm_group_id", "lever_group_id"]
#get parameter years that are defined
param_years_add_sec = [float(x) for x in parameter_table_additional_sectors.columns if x.replace(".", "").isnumeric()]
param_years_add_sec = [int(x) for x in param_years_add_sec if (int(x) == x)]
print("\n\nparam_years_add_sec:")
print(param_years_add_sec)
print(parameter_table_additional_sectors.columns)
print("\n\n")
#initialize for all parameters
all_vals_add_sec = {
    "param_years": param_years_add_sec,
    "future_id": list(range(1, n_lhs + 1)),
    "design_id": list(df_attribute_design_id["design_id"]),
    "time_series_id": list(set(df_attribute_time_series_id["time_series_id"]) & set(parameter_table_additional_sectors["time_series_id"]))
}
#sort some
all_vals_add_sec["time_series_id"].sort()

#loop
for field in fields_add_sec_all_vals:
    #set the field name
    str_field = field.lower().replace(" ", "_")
    #
    if field in ["lever_group_id", "norm_group_id"]:
        set_field = set([x for x in parameter_table_additional_sectors[field] if x > 0])
    else:
        set_field = set(parameter_table_additional_sectors[field])
    #update the dictionary
    all_vals_add_sec.update({str_field: set_field})




###################################
#    GENERATE ATTRIBUTE TABLES    #
###################################

###   RUN ID
df_attribute_run_id_0 = [(x, y) for x in sr.df_strat["strategy_id"] for y in ([0] + all_vals_add_sec["future_id"])]
df_attribute_run_id = pd.DataFrame(df_attribute_run_id_0)
df_attribute_run_id = df_attribute_run_id.rename(columns = {0: "strategy_id", 1: "future_id"})
df_attribute_run_id["run_id"] = range(0, len(df_attribute_run_id))
df_attribute_run_id = df_attribute_run_id[["run_id", "strategy_id", "future_id"]]

if export_ed_files_q:
    #note/export
    print("Exporting run_id attribute to " + sr.fp_csv_attribute_runs)
    df_attribute_run_id.to_csv(sr.fp_csv_attribute_runs, index = None)

###   MASTER ID
df_attribute_master_id = [[x] + [y] + list(z) for x in all_vals_add_sec["design_id"] for y in all_vals_add_sec["time_series_id"] for z in df_attribute_run_id_0]
df_attribute_master_id = pd.DataFrame(df_attribute_master_id)
df_attribute_master_id = df_attribute_master_id.rename(columns = {0: "design_id", 1: "time_series_id", 2: "strategy_id", 3: "future_id"})
fields_df_attribute_master_id = list(df_attribute_master_id.columns)
df_attribute_master_id["master_id"] = list(range(0, len(df_attribute_master_id)))
df_attribute_master_id = df_attribute_master_id[["master_id"] + fields_df_attribute_master_id]
df_attribute_master_id = pd.merge(df_attribute_master_id, df_attribute_run_id, how = "left", on = ["strategy_id", "future_id"])
#fields to order by
fields_ord_dfm = ["master_id", "design_id", "time_series_id", "run_id", "strategy_id", "future_id"]
#reorder
df_attribute_master_id = df_attribute_master_id[fields_ord_dfm].sort_values(by = fields_ord_dfm)

if export_ed_files_q:
    #note/export
    print("Exporting master_id attribute to " + sr.fp_csv_attribute_master)
    #export
    df_attribute_master_id.to_csv(sr.fp_csv_attribute_master, index = None, encoding = "UTF-8")
    #export for gams
    df_attribute_master_id_gams = df_attribute_master_id[["master_id"]].copy().rename(columns = {"master_id": "Escenarios"})
    df_attribute_master_id_gams.to_csv(sr.fp_csv_gams_data_set_escenarios, index = None, encoding = "UTF-8")
    del df_attribute_master_id_gams



#############################
#    GENERATE LHS MATRIX    #
#############################

print("Generating LHS Matrix...")
print("")
#dimensions
p_add_sec = len(all_vals_add_sec["parameter"])
n_add_sec_levers = len(all_vals_add_sec["lever_group_id"])
#generate latin hypercube sample
matrix_lhs = pyd.lhs(p_add_sec + n_add_sec_levers, samples = n_lhs)
#vector for delineating sub-matrices
vec_submat_lhs = [0, p_add_sec, n_add_sec_levers]
#set vector of submatrix index
vec_submat_lhs_names = ["add_sec", "levers"]

##  set names for additional sectors
dict_names_add_sec = [x.lower().replace(" ", "_") for x in all_vals_add_sec["parameter"]]
dict_names_add_sec.sort()
dict_names_add_sec = [[x, dict_names_add_sec[x]] for x in range(0, len(dict_names_add_sec))]
dict_names_add_sec = dict(dict_names_add_sec)

##  set names for lever groups
dict_names_levers = ["lever_group_" + str(x) for x in list(all_vals_add_sec["lever_group_id"])]
dict_names_levers = [[x, dict_names_levers[x]] for x in range(0, len(dict_names_levers))]
dict_names_levers = dict(dict_names_levers)

#initialize the fields for data frames
dict_submat_lhs_fields = {
    "add_sec": dict_names_add_sec,
    "levers": dict_names_levers
}
#setup the output file paths
dict_submat_file_paths = {
    "add_sec": sr.fp_csv_lhs_table_multi_sector,
    "levers": sr.fp_csv_lhs_table_levers
}

#initialize
dict_submat_lhs = {}

if not sr.read_lhs_tables_q:
    print("\n#####\n#####    GENERATING LHS TABLES WITH " + str(n_lhs) + " SAMPLES\n#####\n")

    #break off components
    for i in range(0, len(vec_submat_lhs) - 1):
        #set the field name for the dictionary
        nm = vec_submat_lhs_names[i]
        #check
        if i > -1:
            #get the indeces
            p0 = sum(vec_submat_lhs[0:(i + 1)])
        else:
            p0 = 0
        #set upper limit
        p1 = sum(vec_submat_lhs[0:(i + 2)])
        #temporary dataframe
        df_tmp = pd.DataFrame(matrix_lhs[:, p0:p1], index = None)
        #add name field
        if dict_submat_lhs_fields[nm] != None:
            df_tmp = df_tmp.rename(columns = dict_submat_lhs_fields[nm])
            #set names
            nms = list(df_tmp.columns)
            #add run id
            df_tmp["future_id"] = all_vals_add_sec["future_id"]
            #reorder
            df_tmp = df_tmp[["future_id"] + nms]
        #update dictionary
        dict_submat_lhs.update({nm: df_tmp})

        #export raw lhs data
        if export_ed_files_q:
            #note/export
            print("Exporting LHS for " + nm + " to " + dict_submat_file_paths[nm])
            df_tmp.to_csv(dict_submat_file_paths[nm], index = None)

else:
    #notify
    print("\n#####\n#####    READING IN LHS TABLES\n#####\n")
    #initialize list of "future ids"
    set_future_ids_read = set({})
    ##  initialie booleans

    #initialize the set of futures that are read
    init_sfir_q = True
    #default the imbalance query to fale
    set_imbalance_q = False
    #default exitting to false
    exit_q = False

    #Initialize
    dict_read_futures = {}
    #exit codes
    dict_exit_codes = {
        "set_imbalance": "Number of future_ids in LHS tables are not the same. Check the LHS files to ensure they are using the same future_id indexing.",
        "set_nomatch": "LHS Tables have future ids that do not match specificed number of lhs trials."
    }
    #initialize index
    i = 0
    #read in lhs tables
    while (i < len(vec_submat_lhs_names)) and not set_imbalance_q:
        #get current file
        nm = str(vec_submat_lhs_names[i])
        #get file path
        fp_read = dict_submat_file_paths[nm]
        #read it in
        df_tmp = pd.read_csv(fp_read)
        #reorder it
        #df_tmp = df_tmp[dict_submat_lhs_fields[nm]]
        #update dictionary
        dict_submat_lhs.update({nm: df_tmp})
        #check
        if init_sfir_q:
            #initialize
            set_future_ids_read = set(df_tmp["future_id"])
            #set of futures to compare to for individual exit
            set_future_ids_read_compare = set_future_ids_read
            #turn off initialization
            init_sfir_q = False
        else:
            #current set of Future IDs
            set_future_ids_read_cur = set(df_tmp["future_id"])
            #read in and update the set of intersectional futures
            set_future_ids_read = set_future_ids_read & set_future_ids_read_cur
            #check
            if set_future_ids_read_cur != set_future_ids_read_compare:
                #if any set of futures doesn't match the first one, turn on the exit
                sys.exit(dict_exit_codes["set_imbalance"])
        #notify of successful completion
        print(nm + " LHS table successfully read from " + fp_read)
        #next ieration
        i += 1

    #cut out 0 (some files may have it, some may not)
    set_future_ids_read = set_future_ids_read - set({0})
    #set of what should be the future ids
    set_check_future_ids = set(range(1, n_lhs + 1))
    #compare
    if set_future_ids_read != set_check_future_ids:
        sys.exit(dict_exit_codes["set_nomatch"])

print("\nLHS complete.\n")


##  SET SOME NAMES
fields_ed_add_sec = dict_submat_lhs["add_sec"].columns
fields_ordered_parameters = [x for x in fields_ed_add_sec if x != "future_id"]
sr.print_list_output(fields_ordered_parameters, "fields_ordered_parameters")

#get parameters that are not ramped—they are constant across all years
all_constant_params = set(parameter_table_additional_sectors[parameter_table_additional_sectors["parameter_constant_q"] == 1]["parameter"])
all_constant_params = list(all_constant_params)
all_constant_params.sort()

sr.print_list_output(all_constant_params, "all_constant_params")

#get indices
indices_fop_all_constant_params = [fields_ordered_parameters.index(x) for x in all_constant_params]


#################################################################
###                                                           ###
###    GENERATE EXPERIMENTAL DESIGN FOR ADDITIONAL SECTORS    ###
###                                                           ###
#################################################################

###############################
#    GENERATE LEVER DELTAS    #
###############################

print("Starting generation of lever deltas.")
print("")
##  START BUY BUILDING LONG TABLE OF TRANSFORMED LHS SAMPLES

dsl = dict_submat_lhs["levers"].copy()
dsl = pd.wide_to_long(dsl, i = ["future_id"], j = "lever_group_id", stubnames = "lever_group_")
dsl = dsl.reset_index()
dsl = dsl.rename(columns = {"lever_group_": "lhs_val"})
#data frame out
df_ld_lhs_transformed = []
#loop over design id
for did in all_vals_add_sec["design_id"]:
    #get applicable data
    dict_data = df_attribute_design_id[df_attribute_design_id["design_id"] == did].to_dict()
    #initialize
    lhs_out = dsl.copy()
    #get key for to_dict
    key = list(dict_data["vary_lever_deltas"].keys())[0]
    #get range of values
    vec_vals = np.array(lhs_out["lhs_val"])
    #set header for Future 0
    df_fut_0 = pd.DataFrame([[did, 0, x, 1] for x in all_vals_add_sec["lever_group_id"]], columns = ["design_id", "future_id", "lever_group_id", "lhs_val"])
    #check on varying
    if dict_data["vary_lever_deltas"][key] == 1:
        #transform
        m = float(dict_data["linear_transform_ld_m"][key])
        b = float(dict_data["linear_transform_ld_b"][key])
        #thresholds
        thresh_min = float(dict_data["min_lever_deltas"][key])
        thresh_max = float(dict_data["max_lever_deltas"][key])
        #transformation
        def linear_transform(x):
            return max(min(m*x + b, thresh_max), thresh_min)
        #updated vals
        vec_vals = list(map(linear_transform, vec_vals))
    else:
        #set the deltas to 1
        vec_vals = [1.0 for x in range(len(vec_vals))]
    #update in data frames
    lhs_out["lhs_val"] = vec_vals
    lhs_out["design_id"] = [did for x in range(len(vec_vals))]
    #update
    lhs_out = pd.concat([df_fut_0, lhs_out[["design_id", "future_id", "lever_group_id", "lhs_val"]]])
    #add
    df_ld_lhs_transformed = df_ld_lhs_transformed + [lhs_out]
#convert to dataframee
df_ld_lhs_transformed = pd.concat(df_ld_lhs_transformed)


##  THEN, GO BY STRATEGY TO BUILD DELTAS

print("Starting build of deltas by time series/strategy...")
print("")
#set strategies to build deltas for
strat_lever_deltas = [x for x in all_vals_add_sec["strategy_id"] if x != strat_baseline]
#temporary reduction
ptas_ld = parameter_table_additional_sectors[parameter_table_additional_sectors["lever_group_id"] > 0]

#merge field for generate lever deltas
fields_merge_ld = ["variable_name_lower", "lever_group_id"]
#set extraction fields by type
extraction_fields_ld = fields_merge_ld + [str(x) for x in param_years_add_sec]

#dictionary of lever deltas by strategy
dict_ld = {}
#fields to extract and use in transpose
fields_ext_ld = fields_ordered_parameters
#initialize
df_ld_shaped = []
#loop over time series ids
for ts_id in all_vals_add_sec["time_series_id"]:
    #get baseline data frame
    df_base = ptas_ld[(ptas_ld["strategy_id"] == strat_baseline) & (ptas_ld["time_series_id"] == ts_id)].copy()
    #reduce
    df_base = df_base[extraction_fields_ld]

    #loop over strategies to generate lever deltas by included years
    for strat in strat_lever_deltas:
        #get strategy id
        strat_id = int(strat)#int(sr.dict_strat_ids[strat])
        #get sub data frame
        df_strat = ptas_ld[(ptas_ld["strategy_id"] == strat) & (ptas_ld["time_series_id"] == ts_id)].copy()
        df_strat = df_strat[extraction_fields_ld]
        #data fields
        fields_data = [x for x in extraction_fields_ld if (x not in fields_merge_ld)]
        #get column rename
        dict_rename_df_strat = dict([[str(x), str(strat) + "_" + str(x)] for x in fields_data])
        #update
        df_strat = df_strat.rename(columns = dict_rename_df_strat)
        #merge in
        df_strat = pd.merge(df_strat, df_base, how = "inner", left_on = fields_merge_ld, right_on = fields_merge_ld)
        #set new lever delta fields
        fields_ld = []
        #generate subtraction
        for fd in fields_data:
            field_0 = fd
            field_s = str(strat) + "_" + str(fd)
            field_ld = "ld_" + field_s
            fields_ld = fields_ld + [field_ld]
            #parse out and get difference
            df_strat_tmp = df_strat[[field_0, field_s]].diff(axis = 1)
            #update
            df_strat[field_ld] = df_strat_tmp[field_s]
        #reduce
        df_strat = df_strat[fields_merge_ld + fields_ld]
        #merge in
        df_strat = pd.merge(df_strat, df_ld_lhs_transformed, how = "outer", left_on = ["lever_group_id"], right_on = ["lever_group_id"])
        #build new data
        df_deltas_adj = (np.array(df_strat[fields_ld]).transpose() * np.array(df_strat["lhs_val"])).transpose()
        df_deltas_adj = pd.DataFrame(df_deltas_adj, columns = fields_ld)
        #re-initialize
        df_strat = df_strat[["design_id", "future_id"] + fields_merge_ld]
        df_strat = pd.concat([df_strat, df_deltas_adj], axis = 1, sort = False)
        #update
        dict_ld.update({strat: df_strat})

        # BUILD RESHAPED VALUE

        #loop over design/future
        for did in list(df_strat["design_id"].unique()):
            for fut in list(df_strat["future_id"].unique()):
                df_tmp = df_strat[(df_strat["design_id"] == did) & (df_strat["future_id"] == fut)].copy()
                df_tmp = df_tmp[["variable_name_lower"] + fields_ld]
                #fields to add
                fields_new = list(df_tmp["variable_name_lower"])
                #new data frame
                df_tmp = pd.DataFrame(np.array(df_tmp[fields_ld]).transpose(), columns = fields_new)
                #add year
                df_tmp["year"] = param_years_add_sec
                #add design id and future
                df_tmp["design_id"] = [did for x in range(len(df_tmp))]
                df_tmp["future_id"] = [fut for x in range(len(df_tmp))]
                df_tmp["strategy_id"] = [strat_id for x in range(len(df_tmp))]
                df_tmp["time_series_id"] = [ts_id for x in range(len(df_tmp))]
                #order
                df_tmp = df_tmp[["design_id", "time_series_id", "strategy_id", "future_id", "year"] + fields_new]
                #update
                df_ld_shaped = df_ld_shaped + [df_tmp]
            #notify of completed reshape
            print("Reshaping of LHS table complete for design_id: " + str(did) + ", time_series_id: " + str(ts_id) + ", strategy_id: " + str(strat))

if len(df_ld_shaped) > 0:
    #convert to wide frame
    df_ld_shaped = pd.concat(df_ld_shaped)
    #rename
    df_ld_shaped = df_ld_shaped.rename(columns = dict([[x, "ld_" + x] for x in fields_new]))



##  CREATE RAMP VECTORS FOR UNCERTAINTY

y_0 = int(sr.dict_init["add_sec_variation_start_year"]) - 1
y_1 = max(param_years_add_sec)
y_base = min(param_years_add_sec)
vec_ramp_unc = sr.build_linear_mix_vec((y_0, y_1), (y_base, y_1))
#sr.build_mix_vec(0, 0, 0, "linear")#np.array([max(min((y - y_0)/(y_1 - y_0), 1), 0) for y in param_years_add_sec])
vec_ramp_base = 1 - vec_ramp_unc


##  BUILD BASIC PERCENTAGE CHANGE MATRIX


#for ts_id in all_vals_add_sec["time_series_id"]:
df_ed_baseline = parameter_table_additional_sectors[(parameter_table_additional_sectors["strategy_id"] == strat_baseline) & (parameter_table_additional_sectors["time_series_id"] == 0)].copy()
df_ed_base = df_ed_baseline[extraction_fields_ld].copy()
df_ed_add_sec = dict_submat_lhs["add_sec"]
#initialize array (in order of ext fields)
array_ed_add_sec = np.array(df_ed_add_sec[fields_ordered_parameters])
#order output array
df_max_min = pd.merge(pd.DataFrame(fields_ordered_parameters, columns = ["variable_name_lower"]), df_ed_baseline[["variable_name_lower", "min_2050", "max_2050"]], how = "left", left_on = ["variable_name_lower"], right_on = ["variable_name_lower"])
#generate transformed values of uncertainty
array_ed_add_sec_trans = array_ed_add_sec * np.array(df_max_min["max_2050"]) + (1 - array_ed_add_sec) * np.array(df_max_min["min_2050"])
#add in future 0 (0 change = 100%)
array_ed_add_sec_trans = np.concatenate([np.ones((1, array_ed_add_sec_trans.shape[1])), array_ed_add_sec_trans])
#set data frame of baseline percentage changes
df_perc_change_unc = []

Starting build of components for experimental design.
Check: export_ed_files_q = True
Check: n_lhs = 100


param_years_add_sec:
[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050]
Index(['sector', 'time_series_id', 'strategy_id', 'type', 'parameter',
       'parameter_constant_q', 'min_2050', 'max_2050', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026',
       '2027', '2028', '2029', '2030', '2031', '2032', '2033', '2034', '2035',
       '2036', '2037', '2038', '2039', '2040', '2041', '2042', '2043', '2044',
       '2045', '2046', '2047', '2048', '2049', '2050', 'variable_name_lower',
       'norm_group_id', 'lever_group_id'],
      dtype='object')



Exporting run_id attribute to /Users/jsyme/Documents/Projects/FY20/SWCHE102-1000/git/MultiSector_LTS_Chile/experimental_de

Reshaping of LHS table complete for design_id: 0, time_series_id: 0, strategy_id: 1
Reshaping of LHS table complete for design_id: 1, time_series_id: 0, strategy_id: 1
Reshaping of LHS table complete for design_id: 0, time_series_id: 1, strategy_id: 1
Reshaping of LHS table complete for design_id: 1, time_series_id: 1, strategy_id: 1


In [26]:
#build outcome matrix
for i in range(len(param_years_add_sec)):
    y = param_years_add_sec[i]
    frac_base = vec_ramp_base[i]
    frac_unc = vec_ramp_unc[i]
    #build weighted array
    array_tmp = np.ones(array_ed_add_sec_trans.shape)*(frac_base) + array_ed_add_sec_trans*(frac_unc)
    #NOTE: FOR CONSTANT PARAMS, GET RID OF "RAMP"
    array_tmp[:, indices_fop_all_constant_params] = array_ed_add_sec_trans[:, indices_fop_all_constant_params].copy()
    #convert to data frame
    df_tmp = pd.DataFrame(array_tmp, columns = fields_ordered_parameters)
    #add year
    df_tmp["year"] = [int(y) for x in range(len(df_tmp))]
    df_tmp["future_id"] = [0] + list(df_ed_add_sec["future_id"])
    #df_tmp["time_series_id"] = [ts_id for x in range(len(df_tmp))]
    #organize
    df_tmp = df_tmp[["future_id", "year"] + fields_ordered_parameters]
    #update
    df_perc_change_unc = df_perc_change_unc + [df_tmp]
#build master
df_perc_change_unc = pd.concat(df_perc_change_unc)
df_perc_change_unc = df_perc_change_unc.sort_values(by = ["future_id", "year"])

In [27]:
df_perc_change_unc["medida_biodigestores"].unique()

array([1.        , 0.76149981, 0.88567848, 0.91123552, 0.83531655,
       0.81896796, 0.79317553, 0.7752458 , 0.73407119, 0.97847245,
       1.09644345, 0.84722847, 0.80919952, 1.08186397, 0.94511067,
       0.70619781, 0.83677039, 0.91232671, 0.98219821, 1.08961278,
       0.95654469, 0.75999742, 0.93029581, 1.00594151, 0.72166721,
       0.86052144, 0.72665706, 0.78320479, 1.05059958, 0.82795556,
       1.03759511, 1.02262575, 0.85724724, 1.05971627, 0.85112961,
       1.04349469, 0.93386197, 0.93613298, 0.85346281, 0.99567198,
       1.07799856, 0.8836382 , 0.98896572, 1.01356819, 0.94367058,
       0.75544851, 0.90447088, 0.86500704, 0.76433571, 0.71825394,
       0.87168095, 0.92251938, 0.70189391, 0.99795205, 0.8148212 ,
       0.95090717, 0.89381073, 0.89771269, 1.07242863, 0.74142407,
       1.01768434, 1.06358258, 0.74731685, 1.0468096 , 1.06867034,
       1.01090727, 1.09449355, 0.80370193, 0.97472842, 0.74958509,
       0.84385814, 0.78672528, 0.90253419, 0.7994858 , 0.96424

In [28]:

##  BUILD DATA FRAME BASIS FOR EXPERIMENTAL DESIGN FILE

print("Building data frame basis for experimental design file...")
print("")

if len(df_ld_shaped) > 0:
    #fields that are in the LD matrix
    fields_ld_data = [x for x in df_ld_shaped.columns if (x.replace("ld_", "") in fields_ordered_parameters)]

    #print("\n"*8 + "#"*30)
    #print("\nSUCCESS\n\n")
    #print(df_ld_shaped)
    #print("\n\n" + "#"*30 + "\n"*8)
else:
    fields_ld_data = []
fields_ldparams_data = [x.replace("ld_", "") for x in fields_ld_data]
fields_nonldparams_data = [x for x in fields_ordered_parameters if (x not in fields_ldparams_data)]
#initialize output data frame
df_out = []
#loop over time series
for ts_id in [1]:#all_vals_add_sec["time_series_id"]:
    #get baseline for each time series
    df_ed_baseline = parameter_table_additional_sectors[(parameter_table_additional_sectors["strategy_id"] == strat_baseline) & (parameter_table_additional_sectors["time_series_id"] == ts_id)].copy()
    df_ed_base = df_ed_baseline[extraction_fields_ld].copy()
    #set column headers
    fields_df_ed = list(df_ed_base["variable_name_lower"])
    df_ed_base = df_ed_base.transpose().loc[[str(x) for x in param_years_add_sec],:]
    df_ed_base = df_ed_base.rename(columns = dict([[list(df_ed_base.columns)[x], fields_df_ed[x]] for x in range(len(fields_df_ed))]))
    df_ed_base["year"] = [int(x) for x in df_ed_base.index]

    #loop over design ids
    for did in [0]:#all_vals_add_sec["design_id"]:
        #get applicable data
        dict_data = df_attribute_design_id[df_attribute_design_id["design_id"] == did].to_dict()
        #get key for to_dict
        key = list(dict_data["vary_lever_deltas"].keys())[0]
        #experimental design merge table
        df_ed_merge = df_attribute_master_id[(df_attribute_master_id["time_series_id"] == ts_id) & (df_attribute_master_id["design_id"] == did)]
        list_ed_merge = []
        cols = [x for x in df_ed_merge.columns] + ["year"]
        for y in param_years_add_sec:
            df_tmp = df_ed_merge.copy()
            df_tmp["year"] = [int(y) for x in range(len(df_tmp))]
            df_tmp = df_tmp[cols]
            list_ed_merge = list_ed_merge + [df_tmp]
        #convert
        df_ed_merge = pd.concat(list_ed_merge)
        #add in
        df_ed_merge = pd.merge(df_ed_merge, df_ed_base, how = "outer", left_on = ["year"], right_on = ["year"])
        #renaming dictionary
        dict_rnm = dict([x, "perc_" + x] for x in fields_ordered_parameters)
        #breakout fields
        fields_percs = [dict_rnm[x] for x in fields_ordered_parameters]

        #check if percentages need to be accounted for
        if dict_data["vary_uncertainties"][key] == 1:
            df_tmp = df_perc_change_unc
            #rename to columns
            df_tmp = df_tmp.rename(columns = dict([x, "perc_" + x] for x in fields_ordered_parameters))
            df_ed_merge = pd.merge(df_ed_merge, df_tmp, how = "left", left_on = ["future_id", "year"], right_on = ["future_id", "year"])
            #id fields to breakout on
            fields_id = [x for x in df_ed_merge.columns if (x not in fields_percs) and (x not in fields_ordered_parameters)]
            #seprate out
            df_ed_merge_ids = df_ed_merge[fields_id]
            #convert to product array—has headers fields_ordered_parameters
            array_design = np.array(df_ed_merge[fields_percs]) * np.array(df_ed_merge[fields_ordered_parameters])
        else:
            #id fields to breakout on
            fields_id = [x for x in df_ed_merge.columns if (x not in fields_ordered_parameters)]
            #seprate out
            df_ed_merge_ids = df_ed_merge[fields_id]
            #reduce the design
            array_design = np.array(df_ed_merge[fields_ordered_parameters])
        #update
        df_ed_merge = pd.concat([df_ed_merge_ids, pd.DataFrame(array_design, columns = fields_ordered_parameters)], axis = 1, sort = False)

        #check if lever delta needs to be brought in
        #if dict_data["vary_lever_deltas"][key] == 1:

        # ADD IN LEVER DELTAS

        if len(df_ld_shaped) > 0:
            #fields to merge on
            fields_merge_ld = list(set(df_ed_merge.columns) & set(df_ld_shaped.columns))
        else:
            fields_merge_ld = []

        if len(fields_merge_ld) > 0:
            #merge in lever deltas
            df_ed_merge = pd.merge(df_ed_merge, df_ld_shaped, how = "left", left_on = fields_merge_ld, right_on = fields_merge_ld)
        #split out
        df_ed_merge_ids = df_ed_merge[fields_id + fields_nonldparams_data]
        #add in lever deltas
        array_design = np.array(df_ed_merge[fields_ldparams_data]) + np.array(np.array(df_ed_merge[fields_ld_data].fillna(0)))
        #concatenate
        df_ed_merge = pd.concat([df_ed_merge_ids, pd.DataFrame(array_design, columns = fields_ldparams_data)], axis = 1, sort = False)

        #reduce
        df_ed_merge = df_ed_merge[df_ed_merge["strategy_id"].isin(all_vals_add_sec["strategy_id"])]
        #sort
        df_ed_merge = df_ed_merge.sort_values(by = ["master_id", "year"])
        #clear index
        df_ed_merge = df_ed_merge.reset_index(drop = True)
        #order
        df_ed_merge = df_ed_merge[fields_id + fields_ordered_parameters]

        if len(df_out) == 0:
            #add to output list
            df_out = df_out + [df_ed_merge]
        else:
            df_out = df_out + [df_ed_merge[df_out[0].columns]]

        print("Data frame for time_series_id: " + str(ts_id) + ", design_id: " + str(did) + " complete.")
        print("")

Building data frame basis for experimental design file...

Data frame for time_series_id: 1, design_id: 0 complete.



In [31]:
df_ed_merge["medida_biodigestores"]

0              1
1              1
2              1
3              1
4              1
          ...   
7267    0.731663
7268    0.731663
7269    0.731663
7270    0.731663
7271    0.731663
Name: medida_biodigestores, Length: 7272, dtype: object